In [50]:
import pandas
import geopandas
import geoplot
import re
import osmnx as ox
import matplotlib.pyplot as plt

In [38]:
place_name = "Shorewood, Wisconsin, USA"
graph = ox.graph_from_place(place_name)
area = ox.gdf_from_place(place_name)

/home/imbrius/anaconda3/envs/geo_env/lib/python3.8/site-packages/osmnx/boundaries.py:36: UserWarning: The `boundaries` module has been deprecated and will be removed in a future relase. Use the `geocoder` module's `geocode_to_gdf` function instead.
  warnings.warn(msg)


**Cells above this line should only be run once as they hit public web services.**

In [39]:
def plot_map():
    nodes, edges = ox.graph_to_gdfs(graph)
    fix, ax = plt.subplots()
    area.plot(ax=ax, facecolor="black")
    edges.plot(ax=ax, linewidth=1, edgecolor="#BC8F8F")

In [53]:
def sort_stops(stops):
    stops_block = []
    stops_intersection = []
    
    for i in stops.index:
        stop = stops.loc[i, 'Location']
        
        if stop.find('/') == -1:
            stops_block.append(stops.loc[i])
        else:
            stops_intersection.append(stops.loc[i])
    
    return pandas.DataFrame(stops_block), pandas.DataFrame(stops_intersection)

def prepare_lcn_data(lcn_data):
    stopwords = ['BLK', ',LOT', ',PARK', ',BLDG']
    
    for i in lcn_data.index:
        addr = lcn_data.loc[i, 'Location']
        addr = re.sub(';[A-Z][A-Z]', '', addr)
        
        for word in stopwords:
            addr = addr.replace(word, '')
            
        lcn_data.loc[i, 'Location'] = addr
    
    return sort_stops(lcn_data)

In [70]:
traffic_stops = geopandas.read_file("Copy of Sept 2019 SHPD Traffic Stops.csv")

# For now, we are ignoring the stops_intersection list
# as none of the free geocoders support intersection lookups.
# Google Maps API v3 does but it's a paid subscription service.
# Watch the OSM Nominatim project
# (https://github.com/osm-search/Nominatim/issues/123)
# for when this gets implemented.

stops_block, stops_intersection = prepare_lcn_data(traffic_stops)
lcns = []

for i in stops_block.index:
    print(f"Geocoding address {stops_block.loc[i, 'Location']}")
    
    try:
        lcn = {}
        lcn['Call#'] = stops_block.loc[i, 'Call#']
        lcn['geometry'] = ox.geocoder.geocode(stops_block.loc[i, 'Location'])
    except Exception as ex:
        print(f"Error: {ex}");
    finally:
        lcns.append(lcn)

Geocoding address 1700 E Capitol Dr, Shorewood, WI 53211
Geocoding address 3800 N Maryland Ave, Shorewood, WI 53211
Geocoding address 3300 N Oakland Ave, WI 53211
Geocoding address 4000 N Wilson Dr, Shorewood, WI 53211
Geocoding address 3900 N Prospect Ave, Shorewood, WI 53211
Geocoding address 1900 E Capitol Dr, Shorewood, WI 53211
Geocoding address 3800 N Maryland Ave, Shorewood, WI 53211
Geocoding address 3500 N Oakland Ave, Shorewood, WI 53211
Geocoding address 3800 N Morris Blvd, Shorewood, WI 53211
Geocoding address 3500 N Oakland Ave, Shorewood, WI 53211
Geocoding address 3800 N Maryland Ave, Shorewood, WI 53211
Geocoding address 4000 N Humboldt Ave, Milwaukee, WI 53211
Geocoding address 3600 N Oakland Ave, Shorewood, WI 53211
Geocoding address 1325 E Capitol Dr, Shorewood, WI 53211
Geocoding address 2000 E Capitol Dr, Shorewood, WI 53211
Geocoding address 3900 N Morris Blvd, Shorewood, WI 53211
Geocoding address 3500 N Oakland Ave, Shorewood, WI 53211
Geocoding address 3800 N O

Geocoding address 1300 E Capitol Dr, Shorewood, WI 53211
Geocoding address 1600 E Capitol Dr, Shorewood, WI 53211
Geocoding address 100 E Capitol Dr, Glendale, WI 53212
Error: Nominatim geocoder returned no results for query "100 E Capitol Dr, Glendale, WI 53212"
Geocoding address 4000 N Bartlett Ave, Shorewood, WI 53211
Geocoding address 3900 N Oakland Ave, Shorewood, WI 53211
Geocoding address 3300 N Oakland Ave, Milwaukee, WI 53211
Geocoding address 1800 E Edgewood Ave, Shorewood, WI 53211
Geocoding address 2100 E Capitol Dr, Shorewood, WI 53211
Geocoding address 1500 E Capitol Dr, Shorewood, WI 53211
Geocoding address 4000 N Newhall St, Shorewood, WI 53211
Geocoding address 1000 E Capitol Dr, Milwaukee, WI 53211
Geocoding address 3700 N Oakland Ave, Shorewood, WI 53211
Geocoding address 1600 E Capitol Dr, Shorewood, WI 53211
Geocoding address 4000 N Wilson Dr, Shorewood, WI 53211
Geocoding address 1500 E Capitol Dr, Shorewood, WI 53211
Geocoding address 700 E. Capitol DR; Milwaukee

In [71]:
lcns_series = [Point(xy) for xy in lcns['geometry']]
lcns_series

# merged_frame = stops_block.merge(lcns_series, on='Call#')
# merged_frame

TypeError: list indices must be integers or slices, not str